## Efficient Simulation in the Inventory Model


#### John Stachurski

Simulation can be time consuming, especially if we need to simulate many paths

And it's often true that large sample sizes are necessary to get an accurate picture

Let's look at speeding up our simulations using [Numba](https://numba.pydata.org/)

In [1]:
import numpy as np
from numba import jit, prange, vectorize
from quantecon.util import tic, toc

Parameters:

In [2]:
s, S = 10.0, 100.0
p = 0.4
initial_x = 50.0

### Simulating One Path

Let's look at simulating one path without any optimization.

In [3]:
def sim_one_inventory_path_simple(sim_length=1_000_000):
        
    dvals = np.random.geometric(p, size=sim_length-1) - 1
    X = np.empty(sim_length)
    X[0] = initial_x
    
    for t, d in enumerate(dvals):
        x = X[t]
        if x <= s:
            X[t+1] = max(S - d, 0)
        else:
            X[t+1] = max(x - d, 0)
            
    return X

How fast does this run?

In [4]:
tic()
X = sim_one_inventory_path_simple()
toc()

TOC: Elapsed: 0:00:0.97


0.970921516418457

Now let's build a jitted version.

In [5]:
@jit(nopython=True)
def sim_one_inventory_path_jitted(sim_length=1_000_000):
    
    dvals = np.random.geometric(p, size=sim_length-1) - 1
    X = np.empty(sim_length, dtype=np.int32)
    X[0] = initial_x
    
    for t, d in enumerate(dvals):
        x = X[t]
        if x <= s:
            X[t+1] = max(S - d, 0)
        else:
            X[t+1] = max(x - d, 0)
            
    return X

This is **much** faster:

In [6]:
tic()
X = sim_one_inventory_path_jitted()
toc()

TOC: Elapsed: 0:00:0.28


0.28948092460632324

In [7]:
tic()
X = sim_one_inventory_path_jitted()
toc()

TOC: Elapsed: 0:00:0.07


0.07474923133850098

### Simulating Many Paths

Here's a jitted (and hence fast) piece of code for simulating many paths.

In [8]:
@jit(nopython=True)
def sim_paths_jitted(num_paths=500_000,
                     sim_length=1000):
    
    X = np.empty((num_paths, sim_length), dtype=np.int32)
    X[:, 0] = initial_x
    
    for t in range(sim_length-1):
        dvals = np.random.geometric(p, size=num_paths) - 1
        for m in range(num_paths):
            x, d = X[m, t], dvals[m]
            if x <= s:
                y = max(S - d, 0)
            else:
                y = max(x - d, 0)
            X[m, t+1] = y
    return X


Let's check it's doing what we think it should.

In [9]:
X = sim_paths_jitted(num_paths=6, sim_length=4)

In [10]:
X

array([[50, 48, 45, 32],
       [50, 49, 49, 40],
       [50, 49, 46, 46],
       [50, 47, 45, 44],
       [50, 50, 47, 46],
       [50, 49, 48, 48]], dtype=int32)

Now let's get a reading on the time.

In [11]:
tic()
X = sim_paths_jitted()
toc()

TOC: Elapsed: 0:00:31.85


31.851807355880737

Let's try for more efficient parallelization, where the work is divided into a smaller number of chunks.

In [12]:
@jit(nopython=True, parallel=True)
def sim_paths_jitted_parallel(num_paths=500_000, 
                              sim_length=1000,
                              sections=500):
    
    X = np.empty((num_paths, sim_length), dtype=np.int32)

    section_size = num_paths // sections
    
    for i in prange(sections):
        j = i * section_size
        X[j:j+section_size, :] = sim_paths_jitted(num_paths=section_size, 
                                                sim_length=sim_length)
    return X

In [13]:
X = sim_paths_jitted_parallel(num_paths=8, 
                              sim_length=5, 
                              sections=2)

In [14]:
X

array([[50, 50, 48, 39, 39],
       [50, 47, 46, 43, 43],
       [50, 49, 49, 49, 49],
       [50, 47, 46, 39, 35],
       [50, 50, 49, 47, 44],
       [50, 46, 45, 45, 45],
       [50, 50, 48, 41, 40],
       [50, 49, 49, 49, 46]], dtype=int32)

Now the speed up from parallelization is significant.

In [15]:
tic()
X = sim_paths_jitted_parallel()
toc()

TOC: Elapsed: 0:00:4.92


4.920250415802002